In [1]:
cd ..

C:\Users\shank\Desktop\AUMC\colab\modys-video


# Experiment 1.1 Explore models and approaches for deep learning
Finetune models on left leg amplitude prediction with lying videos as input.
For now this is just to quickly test the settings that Shankara got out of previous experiments, i.e.:

* Cutoff first 50 frames
* Interpolate when likelihood is below 0.7
* Use opposite bodypart if all likelihood is below 0.7 for a bodypart
* Use a standard scaler to scale
    

We don't really know whether this is the best set of hyperparameters. We can further experiment with the following hyperparameters:
* Use a StandardScaler
* Include likelihood
* Use different model architectures
* Use interpolation
* write one function that takes in parameters

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras


from src.helpers import read_scores
from src.data_generators import RawDataGenerator
from src.data_selection import MultipleScoreSelector
from src.ai_func import cross_validation_generator
from src.settings import LYING_VIDEOS_DATA_FOLDER, SITTING_VIDEOS_DATA_FOLDER, DATA_FOLDER


from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU

%load_ext autoreload
%autoreload 2

In [3]:
scores_df = read_scores(DATA_FOLDER / 'data_Scoring_DIS_proximal_trunk_V1.1.xlsx')

## Definitions

In [4]:
SCORES_TO_USE = ['D_LLP_R_tA_pscore']
SCORER_TO_USE = 1
SCORERS_TO_USE = [1, 2, 3]

n_outputs = len(SCORES_TO_USE)

## Pipeline for training a deep neural network

### Define model architecture (here: simple CNN)

In [5]:
def get_model(n_timesteps, n_features, n_outputs):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(n_outputs))
    return model


def get_model_old():
    # simple CNN
    input_layer = keras.layers.Input(shape=(n_timesteps,n_features))
    norm_layer = keras.layers.BatchNormalization()(input_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(norm_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Flatten()(cnn_layer)
    cnn_layer = keras.layers.Dense(100)(cnn_layer)
    output_layer = keras.layers.Dense(n_outputs)(cnn_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

## Train network

In [6]:
def train_model(train_generator, n_timesteps, n_features, n_outputs):
    model = get_model(n_timesteps, n_features, n_outputs)
    model.compile(loss='mae', optimizer=keras.optimizers.Adam())
    model.fit(train_generator, epochs=30)
    return model

def train_cross_val(cross_val):
    y_pred = []
    y_test = []
    splits = []
    for i_split, (train_scores, test_scores) in enumerate(cross_val):
        print(f'Fitting for 5-fold split {i_split}')
        train_generator = RawDataGenerator(train_scores, **data_generation_params)
        scaler = train_generator.get_scaler()
        data_generation_params['scaler'] = scaler
        test_generator = RawDataGenerator(test_scores, **data_generation_params)
        X, y = train_generator.__getitem__(0)
        n_timesteps, n_features = (X.shape[1], X.shape[2])
        model = train_model(train_generator, n_timesteps, n_features, n_outputs)
        y_pred.append(model.predict(test_generator))
        y_test.append(test_scores)
        splits.extend([i_split]*len(test_scores))
        #break # This results in training only for one of the 5 folds
    y_pred = np.vstack(y_pred)
    y_test = pd.concat(y_test)
    return y_test, y_pred, splits

def evaluate(y_test, y_pred):
    results = []
    for i_score, column in enumerate(y_test):
        mae = mean_absolute_error(y_test.iloc[:, i_score], y_pred[:, i_score])
        results.append({'score': column, 'mae': mae})
    return pd.DataFrame(results)

## Test all combinations

In [7]:
results_df = pd.DataFrame(columns=['scorer', 'scores', 'split', 'y', 'pred_y'])

for scorer in SCORERS_TO_USE:
    for cutoff in [0,50]:
        for threshold in [0, 0.7]:
            for scaler in [None, StandardScaler()]:
                for bodyparts in [None, ['ankle1', 'knee1', 'hip1','ankle2', 'knee2', 'hip2']]:
                    data_generation_params = {
                        'videos_folder': LYING_VIDEOS_DATA_FOLDER,
                        'cutoff': cutoff,
                        'interpolation_threshold': threshold,
                        'batch_size': 1, 
                        'scaler': scaler,
                        'bodyparts': bodyparts
                    }  
                    print('params:',data_generation_params)
                    print(f'Training model for scorer {scorer}')
                    selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=scorer)
                    selected_data = selector.transform(scores_df)
                    cross_val = cross_validation_generator(selected_data)
                    y_test, y_pred, splits = train_cross_val(cross_val)
                    for y_item, y_pred_item, split in zip(y_test.values, y_pred, splits):
                        results_df = results_df.append({'scorer':scorer, 
                                                        'scores': ', '.join(SCORES_TO_USE), 
                                                        'split': split, 
                                                        'y':y_item[0], 
                                                        'pred_y': y_pred_item[0], 
                                                        'videos_folder': data_generation_params['videos_folder'],
                                                        'cutoff': data_generation_params['cutoff'],
                                                        'interpolation_threshold': data_generation_params['interpolation_threshold'],
                                                        'batch_size': data_generation_params['batch_size'],
                                                        'scaler': data_generation_params['scaler'],
                                                        'bodyparts': data_generation_params['bodyparts']}, ignore_index=True)

params: {'videos_folder': WindowsPath('C:/Users/shank/Desktop/AUMC/colab/modys-video/data/data_lying_052929'), 'cutoff': 0, 'interpolation_threshold': 0, 'batch_size': 1, 'scaler': None, 'bodyparts': None}
Training model for scorer 1


C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 2s 40ms/step - loss: 70.7553
Epoch 2/30
49/49 [==============================] - 2s 37ms/step - loss: 2.1275
Epoch 3/30
49/49 [==============================] - 2s 37ms/step - loss: 0.5361
Epoch 4/30
49/49 [==============================] - 2s 37ms/step - loss: 0.3074
Epoch 5/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2525
Epoch 6/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2366
Epoch 7/30
49/49 [==============================] - 2s 36ms/step - loss: 0.2331
Epoch 8/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2250
Epoch 9/30
49/49 [==============================] - 2s 36ms/step - loss: 0.2458: 0s - lo
Epoch 10/30
49/49 [==============================] - 2s 36ms/step - loss: 0.2386
Epoch 11/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2359
Epoch 12/30
49/49 [==============================] - 2s 36ms/step - loss: 0.2302


Epoch 10/30
50/50 [==============================] - 2s 36ms/step - loss: 0.2081
Epoch 11/30
50/50 [==============================] - 2s 36ms/step - loss: 0.2125
Epoch 12/30
50/50 [==============================] - 2s 37ms/step - loss: 0.2173
Epoch 13/30
50/50 [==============================] - 2s 36ms/step - loss: 0.2096
Epoch 14/30
50/50 [==============================] - 2s 35ms/step - loss: 0.2166: 0s -
Epoch 15/30
50/50 [==============================] - 2s 35ms/step - loss: 0.2039
Epoch 16/30
50/50 [==============================] - 2s 35ms/step - loss: 0.2113
Epoch 17/30
50/50 [==============================] - 2s 36ms/step - loss: 0.2157
Epoch 18/30
50/50 [==============================] - 2s 35ms/step - loss: 0.2110
Epoch 19/30
50/50 [==============================] - 2s 37ms/step - loss: 0.2082
Epoch 20/30
50/50 [==============================] - 2s 37ms/step - loss: 0.2093
Epoch 21/30
50/50 [==============================] - 2s 36ms/step - loss: 0.2062
Epoch 22/30
50/50 [===

C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


...

Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 5s 94ms/step - loss: 346.8174
Epoch 2/30
36/36 [==============================] - 3s 96ms/step - loss: 146.4317
Epoch 3/30
36/36 [==============================] - 3s 95ms/step - loss: 42.4421
Epoch 4/30
36/36 [==============================] - 3s 95ms/step - loss: 21.6914
Epoch 5/30
36/36 [==============================] - 3s 96ms/step - loss: 58.6391
Epoch 6/30
36/36 [==============================] - 4s 97ms/step - loss: 47.5101
Epoch 7/30
36/36 [==============================] - 3s 95ms/step - loss: 6.3662
Epoch 8/30
36/36 [==============================] - 3s 93ms/step - loss: 3.6507
Epoch 9/30
36/36 [==============================] - 3s 83ms/step - loss: 1.2693
Epoch 10/30
36/36 [==============================] - 4s 111ms/step - loss: 1.6541
Epoch 11/30
36/36 [==============================] - 4s 100ms/step - loss: 0.5848
Epoch 12/30
36/36 [==============================] - 4s 96ms/step - loss: 0.9609


37/37 [==============================] - 2s 49ms/step - loss: 0.9191: 0s - loss: 1
Epoch 10/30
37/37 [==============================] - 2s 48ms/step - loss: 0.6718
Epoch 11/30
37/37 [==============================] - 2s 47ms/step - loss: 0.7100
Epoch 12/30
37/37 [==============================] - 2s 46ms/step - loss: 0.6474
Epoch 13/30
37/37 [==============================] - 2s 47ms/step - loss: 0.4570
Epoch 14/30
37/37 [==============================] - 2s 53ms/step - loss: 0.6187
Epoch 15/30
37/37 [==============================] - 2s 47ms/step - loss: 0.5054
Epoch 16/30
37/37 [==============================] - 2s 47ms/step - loss: 0.4069
Epoch 17/30
37/37 [==============================] - 2s 48ms/step - loss: 0.4135: 0s - loss
Epoch 18/30
37/37 [==============================] - 2s 47ms/step - loss: 0.3527: 0s - l
Epoch 19/30
37/37 [==============================] - 2s 53ms/step - loss: 0.3726
Epoch 20/30
37/37 [==============================] - 2s 48ms/step - loss: 0.3279
Epoch 2

C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 2s 45ms/step - loss: 142.3872: 0s - loss: 
Epoch 2/30
36/36 [==============================] - 2s 48ms/step - loss: 153.8816
Epoch 3/30
36/36 [==============================] - 2s 51ms/step - loss: 15.4348
Epoch 4/30
36/36 [==============================] - 2s 69ms/step - loss: 3.6347
Epoch 5/30
36/36 [==============================] - 2s 47ms/step - loss: 0.6969
Epoch 6/30
36/36 [==============================] - 2s 48ms/step - loss: 0.3108: 1s 
Epoch 7/30
36/36 [==============================] - 2s 47ms/step - loss: 0.3035
Epoch 8/30
36/36 [==============================] - 2s 47ms/step - loss: 0.3062
Epoch 9/30
36/36 [==============================] - 2s 52ms/step - loss: 0.2843
Epoch 10/30
36/36 [==============================] - 2s 51ms/step - loss: 0.2859
Epoch 11/30
36/36 [==============================] - 2s 46ms/step - loss: 0.2623
Epoch 12/30
36/36 [==============================] - 2s 47ms/step - 

37/37 [==============================] - 2s 47ms/step - loss: 2.3279
Epoch 8/30
37/37 [==============================] - 2s 52ms/step - loss: 1.7043
Epoch 9/30
37/37 [==============================] - 2s 48ms/step - loss: 1.1588: 0s - los
Epoch 10/30
37/37 [==============================] - 2s 47ms/step - loss: 1.0736
Epoch 11/30
37/37 [==============================] - 2s 48ms/step - loss: 0.6306: 0s - loss: 0.6
Epoch 12/30
37/37 [==============================] - 2s 48ms/step - loss: 0.8916
Epoch 13/30
37/37 [==============================] - 2s 48ms/step - loss: 0.4869
Epoch 14/30
37/37 [==============================] - 2s 53ms/step - loss: 0.7783
Epoch 15/30
37/37 [==============================] - 2s 48ms/step - loss: 0.4747
Epoch 16/30
37/37 [==============================] - 2s 48ms/step - loss: 0.5834
Epoch 17/30
37/37 [==============================] - 2s 47ms/step - loss: 0.5170
Epoch 18/30
37/37 [==============================] - 2s 47ms/step - loss: 0.5993: 0s - los
Epoch 

In [9]:
results_df.to_excel('results/experiment-1.1-results.xlsx', index=False)

In [11]:
results_df['MAE'] = abs(results_df['y'] - results_df['pred_y'])

In [45]:
results_df.query('cutoff == 50 and scorer==1 and interpolation_threshold==0.7 and scaler!=scaler and bodyparts!=bodyparts')['MAE'].mean()

0.19853577690739785

## Run Once

In [46]:
data_generation_params = {
    'videos_folder': LYING_VIDEOS_DATA_FOLDER,
    'cutoff': 50,
    'interpolation_threshold': 0.7,
    'batch_size': 1, 
    'scaler': None,
    'bodyparts': None
} 

In [47]:
print(f'Training model for scorer {SCORER_TO_USE}')
selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=SCORER_TO_USE)
selected_data = selector.transform(scores_df)
cross_val = cross_validation_generator(selected_data)
y_test, y_pred, splits = train_cross_val(cross_val)

Training model for scorer 1


C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 3s 39ms/step - loss: 74.4381
Epoch 2/30
49/49 [==============================] - 2s 39ms/step - loss: 19.5882
Epoch 3/30
49/49 [==============================] - 2s 39ms/step - loss: 6.4209
Epoch 4/30
49/49 [==============================] - 2s 38ms/step - loss: 1.0732
Epoch 5/30
49/49 [==============================] - 2s 38ms/step - loss: 0.7536
Epoch 6/30
49/49 [==============================] - 2s 38ms/step - loss: 0.3440
Epoch 7/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2683
Epoch 8/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2401
Epoch 9/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2253
Epoch 10/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2227
Epoch 11/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2348
Epoch 12/30
49/49 [==============================] - 2s 40ms/step - loss: 0.2377
Epoch 13

50/50 [==============================] - 3s 50ms/step - loss: 0.2149
Epoch 10/30
50/50 [==============================] - 2s 44ms/step - loss: 0.2135
Epoch 11/30
50/50 [==============================] - 2s 43ms/step - loss: 0.2208
Epoch 12/30
50/50 [==============================] - 2s 40ms/step - loss: 0.2064
Epoch 13/30
50/50 [==============================] - 2s 44ms/step - loss: 0.2214
Epoch 14/30
50/50 [==============================] - 2s 45ms/step - loss: 0.2168
Epoch 15/30
50/50 [==============================] - 2s 39ms/step - loss: 0.2111
Epoch 16/30
50/50 [==============================] - 2s 39ms/step - loss: 0.2155
Epoch 17/30
50/50 [==============================] - 2s 41ms/step - loss: 0.2106
Epoch 18/30
50/50 [==============================] - 2s 42ms/step - loss: 0.2165
Epoch 19/30
50/50 [==============================] - 2s 44ms/step - loss: 0.2076
Epoch 20/30
50/50 [==============================] - 2s 45ms/step - loss: 0.2145
Epoch 21/30
50/50 [=====================

In [48]:
print(evaluate(y_test, y_pred))

               score       mae
0  D_LLP_R_tA_pscore  0.210181
